In [11]:
import pandas as pd
import ta

In [12]:
df = pd.read_csv('../BTC_ETH_1week_1m.csv')

In [13]:
symbols = ['ETH']

for symbol in symbols:
    df[f'{symbol}_RSI'] = ta.momentum.rsi(close=df[f'{symbol}-USD_Close'], window=14)

df.dropna(inplace=True)

In [14]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, symbols):
        self.short_unit = None
        self.short_amount = 0
        self.short_price = 0
        self.long_price = 0


        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.symbols = symbols
        
        self.buys = []
        self.sells = []
        
        self.wins = 0
        self.losses = 0
        self.ties = 0
        
        self.trading_fee_multiplier = 0.99945      # VIP level 0, paying fees with BNB 0.99975
        self.number_of_buys = len(self.buys)

    def buy_long(self, symbol, long_price, time):
        self.balance_amount = (self.balance_amount / long_price) * self.trading_fee_multiplier
        self.long_price = long_price
        self.balance_unit = symbol
        self.buys.append([symbol, time, long_price, 'long'])
        self.number_of_buys = len(self.buys)
        print('filled long', long_price, time, '\n\n')

    def buy_short(self, symbol, short_price, atr, time):
        self.balance_amount = self.balance_amount * self.trading_fee_multiplier
        self.short_amount = (self.balance_amount / short_price) 
        self.short_price = short_price
        self.short_unit = symbol
        self.buys.append([symbol, time, short_price, 'short'])
        self.number_of_buys = len(self.buys)
        self.sl = short_price + (env.slm*atr)
        print('filled short', short_price, time, '\n\n')

    def sell_long(self, sell_price, time):
        self.balance_amount = (self.balance_amount * sell_price) * self.trading_fee_multiplier
        self.sells.append([symbol, time, sell_price])
        self.balance_unit = 'BUSD'
    
    def sell_short(self, sell_price, time):
        self.balance_amount = (self.balance_amount + (self.short_amount * ((self.short_price*self.trading_fee_multiplier) - sell_price)))
        self.sells.append([symbol, time, sell_price])
        self.short_unit = None
        self.short_amount = 0
    


In [15]:
"""
BUY:
    Long: 
        1. Check if symbol crosses under 30 on RSI -> Buy

    Short:
        1. Check if symbol crosses over 70 on RSI

"""

'\nBUY:\n    Long: \n        1. Check if symbol crosses under 30 on RSI -> Buy\n\n    Short:\n        1. Check if symbol crosses over 70 on RSI\n\n'

In [16]:
env = TradingEnv(balance_amount=100, balance_unit='BUSD', symbols=symbols)
for i in range(len(df)):    
    print(env.balance_amount, env.balance_unit, env.number_of_buys)
    if env.balance_unit == 'BUSD' and env.short_unit == None:  # Want to buy
        for symbol in symbols:

            ######## LONG           (lower + ((upper-lower)*multiplier))
            if df[f'{symbol}_RSI'].iloc[i] <= 30:
                env.buy_long(symbol, long_price=df[f'{symbol}-USD_Close'].iloc[i],time=df['OpenTime'].iloc[i])
                break
    

            ######## SHORT
            # if df[f'{symbol}_RSI'].iloc[i] >= 70:
            #     env.buy_short(symbol, short_price=df[f'{symbol}-USD_Close'].iloc[i], atr=df[f'{symbol}_ATR'].iloc[i], time=df['OpenTime'].iloc[i])
            #     break



    ####################################################################################################################

    ############## Want to sell long position
    elif env.balance_unit != 'BUSD':

        # Sell long for profit
        if df[f'{symbol}_RSI'].iloc[i] >= 70:
            print('sell long', df[f'{env.balance_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.wins += 1
            env.sell_long(sell_price=df[f'{env.balance_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            continue
    

    ############## Want to sell short position
    
    # else:
    #     print(env.short_amount)
    #     # Sell short for profit
    #     if env.cross_down[env.short_unit] == True and df[f'{env.short_unit}-USD_Close'].iloc[i] >= (df[f'{symbol}_Lower_Band'].iloc[i] + ((df[f'{symbol}_Upper_Band'].iloc[i]-df[f'{symbol}_Lower_Band'].iloc[i]) * env.band_stop_run)):
    #         print('sell short', df[f'{env.short_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
    #         env.wins += 1
    #         env.sell_short(sell_price=df[f'{env.short_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
    #         env.reset_crosses(symbols)
    #         continue
        
    #     # Sell short at stop loss
    #     elif df[f'{env.short_unit}-USD_Close'].iloc[i] >= env.sl:
    #         print('Stop loss hit: sell short', df[f'{env.short_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
    #         env.losses += 1
    #         env.sell_short(sell_price=df[f'{env.short_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
    #         env.reset_crosses(symbols)
    #         continue
        
    #     # Sell short at break-even  
    #     if env.sma_cross_down[env.short_unit] == True and df[f'{env.short_unit}-USD_High'].iloc[i] >= env.short_price:
    #         print('Break even hit: sell short', env.short_price, df['OpenTime'].iloc[i], '\n\n')
    #         env.ties += 1
    #         env.sell_short(sell_price=env.short_price, time=df['OpenTime'].iloc[i])
    #         env.reset_crosses(symbols)
    #         continue

    #     # Check if SMA is crossed down
    #     if df[f'{env.short_unit}-USD_Low'].iloc[i] <= df[f'{env.short_unit}_SMA'].iloc[i]:
    #         print('SMA short crossed')
    #         env.sma_cross_down[env.short_unit] = True

    #     # Check if lower band is crossed
    #     if df[f'{env.short_unit}-USD_Low'].iloc[i] <= df[f'{env.short_unit}_Lower_Band'].iloc[i]:
    #         print('short crossed')
    #         env.cross_down[env.short_unit] = True
    




if env.balance_unit != 'BUSD':
    env.sell_long(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
if env.short_unit != None:
    env.sell_short(df[f'{env.short_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
print(env.balance_amount, env.balance_unit, env.number_of_buys, env.wins, env.ties, env.losses)

100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
filled long 2547.1 2022-01-28 20:24:00 


0.039238742098857526 ETH 1
0.039238742098857526 ETH 1
0.039238742098857526 ETH 1
0.039238742098857526 ETH 1
0.039238742098857526 ETH 1
0.039238742098857526 ETH 1
0.03

In [17]:

data = pd.DataFrame([(i[1] for i in env.buys), (j[2] for j in env.buys), (a[3] for a in env.buys), (k[1] for k in env.sells), (l[2] for l in env.sells)])
data

,0,1,2,3,4
0,2022-01-28 20:24:00,2022-01-29 00:17:00,2022-01-29 04:03:00,2022-01-29 09:37:00,2022-01-29 17:03:00
1,2547.1,2529.56,2541.66,2579.22,2605.65
2,long,long,long,long,long
3,2022-01-28 23:06:00,2022-01-29 01:31:00,2022-01-29 04:54:00,2022-01-29 11:58:00,2022-01-29 18:58:00
4,2546.0,2548.21,2551.29,2558.87,2601.4
